In [11]:
import importlib 
import textoABraille as tb
from PIL import Image

import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'D:\Tesseract\tesseract.exe' 

In [17]:
importlib.reload(tb) #Para actualizar los cambios en el archivo textoABraille.py

<module 'textoABraille' from 'd:\\Repositorios\\BrailleFabcore\\Traduccion\\textoABraille.py'>

In [16]:
ejemplo="Hola, Pepino"
print(tb.convertirTextoABraille(ejemplo))

⠓⠕⠇⠁⠂ ⠏⠑⠏⠊⠝⠕


In [20]:

braille_signals = tb.convertirASeñales("Hola, Pepino")
binary_signals = [format(signal, '06b') for signal in braille_signals]
print(binary_signals)

['110010', '101010', '111000', '100000', '010000', '000000', '111100', '100010', '111100', '010100', '101110', '101010']
